In [13]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="App_Benesh_")


latitude=40.7423543
longitude=-73.98915076

In [14]:
location = geolocator.reverse(f"{latitude},{longitude}")

In [15]:
# print(location.address)

# print((location.latitude, location.longitude))

print(location.raw)

{'place_id': 331978482, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'node', 'osm_id': 10177726977, 'lat': '40.7423907', 'lon': '-73.9891937', 'class': 'amenity', 'type': 'waste_basket', 'place_rank': 30, 'importance': 9.175936522464359e-05, 'addresstype': 'amenity', 'name': '', 'display_name': 'Broadway, Flatiron District, Manhattan Community Board 5, Manhattan, New York County, City of New York, New York, 10019, United States', 'address': {'road': 'Broadway', 'neighbourhood': 'Flatiron District', 'borough': 'Manhattan', 'county': 'New York County', 'city': 'City of New York', 'state': 'New York', 'ISO3166-2-lvl4': 'US-NY', 'postcode': '10019', 'country': 'United States', 'country_code': 'us'}, 'boundingbox': ['40.7423407', '40.7424407', '-73.9892437', '-73.9891437']}


In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
        .appName("spark-etl_nybike_bronze") \
            .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/06/17 10:21:09 WARN Utils: Your hostname, DESKTOP-2ELJ47S resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/06/17 10:21:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/17 10:21:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
df= spark.read.csv('volumes_and_data_services/data_nybike/2013-citibike-tripdata/6_June/201306-citibike-tripdata_1.csv',header=True)

25/05/20 17:18:56 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [10]:
df.count()

577703

In [17]:
df = df.withColumnsRenamed({'start station latitude':'start_latitude','start station longitude':'start_longitude','end station latitude':'end_latitude','end station longitude':'end_longitude'})

In [4]:
from pyspark.sql.functions import to_timestamp,col
df=df.withColumn('starttime',to_timestamp(col('starttime')))

In [ ]:
import sys
import os

# Add the src directory to the Python path
sys.path.append(os.path.abspath("src/jobs/pyspark"))

25/06/17 10:21:32 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [5]:
from src.jobs.pyspark.interfaces import *
from src.jobs.pyspark.transformers import *
from pyspark.sql.functions import lit
df = AddColumnWithLiteralValue().run(df,{'column_to_add':{'column_name':'genre_enr','column_value':'FEMMEUILE'}})

Column With value literal initiated


In [ ]:
import uuid
from src.jobs.pyspark.transformers import AddDimensionsForTimes

In [9]:
import uuid
from pyspark.sql.functions import udf,coalesce,when,col,lit,hash,datediff,year,quarter,dayofmonth,month,dayofweek,date_format,concat,to_timestamp,udf,isnull ,when

@udf
def uuid_gen():
    return str(uuid.uuid4())

# df.select('*',uuid_gen().alias('id_uuid')).show(10)
df.withColumn('id_uuid',uuid_gen()).show(10)

+------------+-------------------+-------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+--------------------+
|tripduration|          starttime|           stoptime|start station id|  start station name|start station latitude|start station longitude|end station id|    end station name|end station latitude|end station longitude|bikeid|  usertype|birth year|gender|             id_uuid|
+------------+-------------------+-------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+--------------------+
|         695|2013-06-01 00:00:01|2013-06-01 00:11:36|             444|  Broadway & W 24 St|            40.7423543|           -73.98915076|         434.0|     9 Ave & W 18 

In [7]:
from src.jobs.pyspark.interfaces import *
from src.jobs.pyspark.transformers import AddDimensionsForTimesv2
from pyspark.sql.functions import *
config={'datetime_column':'starttime'}
# df.show(1)
AddDimensionsForTimesv2().run(df,config).show(5)


Dimensions time column add initiated
+------------+-------------------+-------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+---------+----+-----+-------+---+-------+----------+------------+------------+
|tripduration|          starttime|           stoptime|start station id|  start station name|start station latitude|start station longitude|end station id|    end station name|end station latitude|end station longitude|bikeid|  usertype|birth year|gender|genre_enr|year|month|quarter|day|weekday|month_name|weekday_name|quarter_name|
+------------+-------------------+-------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+---------+----+-----+-------+---+-------+--

In [15]:
from src.jobs.pyspark.interfaces import *
from src.jobs.pyspark.transformers import AddUuidToColumnID
from pyspark.sql.functions import *
config={'column_id':'uuid_id'}
# df.show(1)
AddUuidToColumnID().run(df,config).show(5)

Add uuid in to th column: uuid_id
+------------+-------------------+-------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+---------+--------------------+
|tripduration|          starttime|           stoptime|start station id|  start station name|start station latitude|start station longitude|end station id|    end station name|end station latitude|end station longitude|bikeid|  usertype|birth year|gender|genre_enr|             uuid_id|
+------------+-------------------+-------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+---------+--------------------+
|         695|2013-06-01 00:00:01|2013-06-01 00:11:36|             444|  Broadway & W 24 St|            40.7

In [19]:
import uuid
from pyspark.sql.functions import udf,coalesce,when,col,lit,hash,datediff,year,quarter,dayofmonth,month,dayofweek,date_format,concat,to_timestamp,udf,isnull ,when,col
from geopy.geocoders import Nominatim

# geolocator = Nominatim(user_agent="App_Benesh_")

@udf
def enrich_location(longitude,latitude): 
    geolocator = Nominatim(user_agent="App_Benesh_")
    location = geolocator.reverse(f"{latitude},{longitude}")
    return location

In [25]:
df = df.select('*',enrich_location(col("start_latitude"),col("start_longitude")).alias("enr_start_location"), enrich_location(col("end_latitude"),col("end_longitude")).alias("enr_end_location"))

In [6]:
# df.show(10)

25/06/16 17:04:04 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

In [9]:
from chispa.dataframe_comparer import *
from pyspark.testing.utils import assertDataFrameEqual
from pyspark.sql.types import StructType, StructField, StringType, IntegerType , TimestampType , DoubleType , FloatType, DateType
from pyspark.sql.functions import udf,coalesce,when,col,lit,year,quarter,dayofmonth,month,dayofweek,date_format,concat,to_timestamp,udf,isnull ,when

from interfaces import DataTransformer
from transformers import RenameColumn,AddColumnDiffTime,DropColumns,CastToTimestamp
from model_data import bronze_schema_ny_bike
from helpers_utils import config_reader

input_data = [{
            "start_at":"1/1/2015 0:01",
            "stop_at":"1/1/2015 0:24"
            }]

expected_data = [{
            "start_at":"2015-1-01 0:01:00",
            "stop_at":"2015-1-01 0:24:00"             
            }]


config={
    "cast_to_timestamp":["start_at","stop_at"]
}
transformer_cast = CastToTimestamp()
df_input_data=spark.createDataFrame(input_data)
df_expected_data=spark.createDataFrame(expected_data)


# assertDataFrameEqual(result_transformation,df_expected_data)

In [10]:
df_expected_data = df_expected_data\
    .withColumn('start_at',to_timestamp(col('start_at')))\
    .withColumn('stop_at',to_timestamp(col('stop_at')))
result_transformation = transformer_cast.run(df_input_data,config)

Cast to timestamp: ['start_at', 'stop_at']


In [8]:
df_input_data.show()

+--------------+--------------+
|      start_at|       stop_at|
+--------------+--------------+
|,1/1/2015 0:01|,1/1/2015 0:24|
+--------------+--------------+



In [11]:
result_transformation.show()

+-------------------+-------------------+
|           start_at|            stop_at|
+-------------------+-------------------+
|2015-01-01 00:01:00|2015-01-01 00:24:00|
+-------------------+-------------------+

